In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing required libraries 

In [ ]:
eff_b0_path = "../input/efficientnet-pytorch/efficientnet-b0-08094119.pth"
package_path = "../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/"
import sys 
sys.path.append(package_path)
import sys 
import os 
import glob 
import time 
import random 
import numpy as np 
import pandas as pd 
import pydicom 
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2 

import torch 
from torch import nn 
from torch.utils import data as torch_data 
from torch.nn import functional as F 

import efficientnet_pytorch

from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm 

# device selection 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
img_size = 256 
n_frames = 10 
cnn_features = 256 
lstm_hidden = 32 
n_fold = 5 
n_epochs = 10 

# creating the model 

In [ ]:
device

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.map = nn.Conv2d(in_channels = 4 , out_channels = 3 , kernel_size = 1 )
        self.net = efficientnet_pytorch.EfficientNet.from_name("efficientnet-b0")
        checkpoint = torch.load('../input/efficientnet-pytorch/efficientnet-b0-08094119.pth')
        self.net.load_state_dict(checkpoint)
        n_features = self.net._fc.in_features 
        self.net._fc = nn.Linear(in_features = n_features , out_features = cnn_features)
    
    def forward(self, x ):
        x = F.relu(self.map(x))
        out = self.net(x)
        return out 

    
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cnn = CNN()
        self.rnn = nn.LSTM(cnn_features , lstm_hidden , 2, batch_first = True)
        self.fc = nn.Linear(lstm_hidden, 1, bias = True)
        
    def forward(self, x):
        batch_size , timesteps, C, H, W = x.size() 
        c_in = x.view(batch_size* timesteps , C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, timesteps, -1)
        output , (hn,cn) = self.rnn(r_in)
        out = self.fc(hn[-1])
        return out

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
        
    data = np.float32(cv2.resize(data, (CFG.img_size, CFG.img_size)))
    return torch.tensor(data)

def load_dicom_line(path):
    t_paths = sorted(
        glob.glob(os.path.join(path, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    images = []
    for filename in t_paths:
        data = load_dicom(filename)
        if data.max() == 0:
            continue
        images.append(data)
        
    return images

In [ ]:
class DataRetrival(Dataset):
    def __init__(self, paths,targets , transform = None):
        self.paths = paths 
        self.transform = transform
        self.targets = targets
    
    def __len__(self):
        return len(self.paths)
    
    def read_in(self,image_paths):
        total_img = [load_dicom(path) for path in image_paths]
        
        if len(total_img) == 0 :
            total_img = torch.zeros(n_frames , img_size , img_size)
        
        else: 
            total_img = torch.stack(total_img)
        
        return total_img
    
    def __getitem__(self, index):
        _id = self.paths[index]
        patient_path = f"../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(_id).zfill(5)}/"
        channels = [] 
        for t in ["FLAIR","T1w", "T1wCE", "T2w"]:
            t_paths = sorted(
                glob.glob(os.path.join(patient_path, t, "*")), 
                key=lambda x: int(x[:-4].split("-")[-1]),
            )
            
            num_samples = n_frames 
            if len(t_paths) < num_samples:
                in_frames_path = t_paths 
            else:
                in_frames_paths = uniform_temporal_subsample(t_paths , num_samples)
            
            channel = self.read_in(in_frames_path)
            if channel.shape[0] == 0 :
                print('1 channel empty')
                channel = torch.zeros(num_samples , img_size, img_size)
            
            channels.append(channel)
        channels = torch.stack(channels).transpose(0,1)
        
        y = torch.tensor(self.targets[index] , dtype = torch.float)
        return {"X" : channels.float(), "y": y}

In [ ]:
df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")

In [ ]:
df

In [ ]:
train_loader = torch_data.DataLoader(
    DataRetrival(df['BraTS21ID'].values,
                df['MGMT_value'].values
                ),
    batch_size = 8 , 
    num_workers = 4, 
    shuffle= True
)

# creating the training loop 


In [ ]:
def train_loop(epochs , model , optimizer , dataloader , loss):
    model = model.to(device)
    if optimizer == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model.train()
    
    for epoch in tqdm(range(epochs)):
        for step, data in enumerate(dataloader):
            X = data['X'].to(device)
            targets = data['y'].to(device)
            optimizer.zero_grad()
            outputs = model(X).squeeze(1)
            data_loss = loss(outputs, targets)
            data_loss.backward()
            
            optimizer.step()
        
        if (epoch%3 == 0):
            print(f'epoch no {epoch+1} completed at {time.time()}')

In [ ]:
model = Model()
train_loop(epochs = n_epochs ,
           model = model ,
           optimizer = "adam" ,
           dataloader = train_loader,
           loss = F.binary_cross_entropy_with_logits)


In [ ]:
torch.save(model,"./efficient_net_b0_model.pth" )